In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

from elftools.elf.elffile import ELFFile

from depsurf import BuildVersion, Functions, disable_dwarf_cache


def dump_dwarf_smoke(vmlinux_path: Path, smoke_path: str, debug: bool):
    with vmlinux_path.open("rb") as f:
        dwarfinfo = ELFFile(f).get_dwarf_info(relocate_dwarf_sections=False)
        cu = next(
            cu
            for cu in dwarfinfo.iter_CUs()
            if cu.get_top_DIE().get_full_path().endswith(smoke_path)
        )
        funcs = Functions.from_cus([cu], debug=debug)
        return funcs.df


# dump_dwarf_smoke(
#     BuildVersion.from_str("5.19.0-21-generic-amd64").vmlinux_path,
#     smoke_path="head_64.S",
#     debug=True,
# )

In [4]:
def dump_dwarf(vmlinux_path: Path, result_path: Path):
    disable_dwarf_cache()

    if result_path.exists():
        print(f"Skipping {result_path}")
        return

    with vmlinux_path.open("rb") as f:
        dwarfinfo = ELFFile(f).get_dwarf_info(relocate_dwarf_sections=False)
        funcs = Functions.from_dwarfinfo(dwarfinfo)
        funcs.dump(path=result_path)

for v in BuildVersion.filter():
    dump_dwarf(v.vmlinux_path, v.dwarf_path)

Skipping /Users/szhong/Downloads/bpf-study/data/dwarf/4.4.0-21-generic-amd64.pkl
Skipping /Users/szhong/Downloads/bpf-study/data/dwarf/4.8.0-22-generic-amd64.pkl
Skipping /Users/szhong/Downloads/bpf-study/data/dwarf/4.10.0-19-generic-amd64.pkl
Skipping /Users/szhong/Downloads/bpf-study/data/dwarf/4.13.0-16-generic-amd64.pkl
Skipping /Users/szhong/Downloads/bpf-study/data/dwarf/4.15.0-20-generic-amd64.pkl
Skipping /Users/szhong/Downloads/bpf-study/data/dwarf/4.18.0-10-generic-amd64.pkl
Skipping /Users/szhong/Downloads/bpf-study/data/dwarf/5.0.0-13-generic-amd64.pkl
Skipping /Users/szhong/Downloads/bpf-study/data/dwarf/5.3.0-18-generic-amd64.pkl
Skipping /Users/szhong/Downloads/bpf-study/data/dwarf/5.4.0-26-generic-amd64.pkl
Skipping /Users/szhong/Downloads/bpf-study/data/dwarf/5.8.0-25-generic-amd64.pkl
Skipping /Users/szhong/Downloads/bpf-study/data/dwarf/5.11.0-16-generic-amd64.pkl
Skipping /Users/szhong/Downloads/bpf-study/data/dwarf/5.13.0-19-generic-amd64.pkl
Skipping /Users/szhong

In [ ]:
import multiprocessing as mp

with mp.Pool(mp.cpu_count()) as pool:
    pool.starmap(
        dump_dwarf,
        [(v.vmlinux_path, v.dwarf_path) for v in BuildVersion.filter()],
    )